In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
test_ID = test.Id
y = train.SalePrice
qualitative = list(train.columns[train.dtypes == "object"])
quantitative = list(test.select_dtypes(include=['float64',"int64"]).columns)
quantitative.remove("Id")




In [2]:
cols = ['FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold']
train[cols] = train[cols].fillna(0)
test[cols] = test[cols].fillna(0)
old_cols = [x for x in test.columns if x not in cols]

In [3]:
key = pd.DataFrame(columns=["index"], index=pd.MultiIndex.from_tuples([], names=['variable','value']))

for x in cols:
    kf = train.groupby(x)["SalePrice"].mean().to_frame().sort_values("SalePrice")
    kf = kf.reset_index().reset_index().drop(columns=["SalePrice"])
    kf["variable"] = x
    kf.index = kf.index + 1
    key = pd.concat([key, kf.rename(columns={x:"value"}).set_index(["variable","value"])])



In [4]:
train = train.drop("SalePrice", axis=1)
features = pd.concat([train, test])

reshape_df = pd.melt(features, id_vars="Id", value_vars=cols).merge(key, on=["variable","value"])
qualDf= reshape_df.pivot(index='Id', columns='variable')["index"]
#qualDf.columns = [str(x)+"_E" for x in qualDf.columns]


features = qualDf.merge(features[old_cols], on=["Id"])

In [5]:
#features = features.drop(features[features[cols].isnull().any(axis=1)].index)
features[cols] = features[cols].fillna(0)
features[cols] = features[cols].astype('int')

In [6]:
features = features.drop(columns=["Id"])



In [7]:
from scipy.stats import norm, skew
numeric_feats = features.dtypes[features.dtypes != "object"].index

# Check the skew of all numerical features
skewed_feats = features[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
print("\nSkew in numerical features: \n")
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness.head(15)


Skew in numerical features: 



,Skew
MiscVal,21.947195
PoolQC,20.723994
PoolArea,16.898328
LotArea,12.822431
LowQualFinSF,12.088761
3SsnPorch,11.376065
LandSlope,4.975157
KitchenAbvGr,4.302254
BsmtFinSF2,4.145323
EnclosedPorch,4.003891


In [8]:
skewness = skewness[abs(skewness) > 0.75]
print("There are {} skewed numerical features to Box Cox transform".format(skewness.shape[0]))

from scipy.special import boxcox1p
skewed_features = skewness.index
lam = 0.15
for feat in skewed_features:
    features[feat] = boxcox1p(features[feat], lam)

There are 58 skewed numerical features to Box Cox transform


In [9]:
features[quantitative] = features[quantitative].fillna(0)

In [10]:
features = pd.get_dummies(features)

In [11]:
train = features[:len(y)]
test = features[len(y):]

In [12]:
y_train = np.log1p(y)

In [13]:
from tpot import TPOTRegressor
## It's a implementation of some customized models to do in future
tpot_config = {
    'sklearn.ensemble.LassoRegressor': {
        ''
    },
    'xgboost.XGBRegressor': {
        'alpha': [1e-3, 1e-2, 1e-1, 1., 10., 100.],
        'fit_prior': [True, False]
    },
    'sklearn.naive_bayes.MultinomialNB': {
        'alpha': [1e-3, 1e-2, 1e-1, 1., 10., 100.],
        'fit_prior': [True, False]
    }
}
tpot = TPOTRegressor(verbosity=2, scoring='r2', cv=3, 
                      n_jobs=-1, generations=6, config_dict='TPOT light',
                      population_size=50, random_state=3,
                      early_stop = 5)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


In [14]:
tpot.fit(train, y_train)

Optimization Progress:   0%|          | 0/350 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.8924553266813899

Generation 2 - Current best internal CV score: 0.892462122884362

Generation 3 - Current best internal CV score: 0.892462122884362

Generation 4 - Current best internal CV score: 0.892462122884362

Generation 5 - Current best internal CV score: 0.8925964331524074

Generation 6 - Current best internal CV score: 0.8931802198215261

Best pipeline: LassoLarsCV(DecisionTreeRegressor(input_matrix, max_depth=3, min_samples_leaf=11, min_samples_split=13), normalize=False)


TPOTRegressor(config_dict='TPOT light', cv=3, early_stop=5, generations=6,
              n_jobs=-1, population_size=50, random_state=3, scoring='r2',
              verbosity=2)

In [50]:

predictions = (tpot.predict(test))
y_test = np.log1p(pd.read_csv("../input/full-score.csv")["SalePrice"])
rmse = np.sqrt(MSE(y_test, predictions)) 
print("RMSE : % f" %(rmse)) 


RMSE :  0.130294


In [51]:
predictions

array([11.69240266, 11.99078643, 12.12604181, ..., 11.982316  ,
       11.71499724, 12.34187075])

In [52]:
y_test

0       11.561725
1       12.055256
2       12.154258
3       12.183321
4       12.162648
          ...    
1454    11.413116
1455    11.170449
1456    11.782960
1457    11.790565
1458    12.144203
Name: SalePrice, Length: 1459, dtype: float64

In [ ]:
sub = pd.DataFrame()
sub['Id'] = test_ID
sub['SalePrice'] = finalMd
sub.to_csv('../output/tpot_submission.csv',index=False)